# **자연어 2일 - Focused Crawling**
- **Bs4** 를 활용한 DOM 살펴보기
- **Selenium** 을 활용한 **Session 과 Cookie** 의 활용
- 간단한 표준작업 Process 를 정리해보자

## **1 DOM 객체의 Focusing**
**bs4 객체** 에서 **.find() .find_all()** 을 활용한 수집
```python
Signature:
bs4obj.find_all(
    name  = None,
    attrs = {},
    recursive = True,  # 자손까지 찾는다
    text  = None,
    limit = None,  # 추출할 객체의 갯수
    **kwargs,
)
```

In [1]:
# 앞에서 작업한 수집모듈 
import requests, time
def download(method, url, params=None, data=None):
    userAgent = {"user-agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
    resp = requests.request(method, url, params=params, data=data, headers= userAgent)
    return resp

In [2]:
# 뉴스 Url 을 활용한 자료수집
url = "https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=015&aid=0004180601"

from bs4 import BeautifulSoup
html = download('get', url)
dom = BeautifulSoup(html.text, "lxml") # 오류시 "html.parser"
type(html.text), type(dom)

(str, bs4.BeautifulSoup)

In [3]:
# 메서드를 활용한 Focusing Crawling
print(dom.html.head.title)  # DOM tree를 활용한 포커싱
print(dom.a)                # Anchor Text를 활용한 포커싱
print(dom.a.attrs)          # 포커싱 DOM 의 속성내용 
print(dom.a["href"])        # 속성 중 필요한 값 추출
print(dom.a.text)           # 텍스트 내용만 추출

<title>'브렉시트 강경파' 보리스 존슨 英 새 총리 오르다 : 네이버 뉴스</title>
<a href="#lnb" tabindex="1"><span>메인 메뉴로 바로가기</span></a>
{'href': '#lnb', 'tabindex': '1'}
#lnb
메인 메뉴로 바로가기


In [4]:
# .find(), find_all() 함수를 활용한 Focused Crawling
print(dom.find("a"))
print(dom.find_all("a")[:4])
type(dom.find("a")), type(dom.find_all("a")), len(dom.find_all("a"))

<a href="#lnb" tabindex="1"><span>메인 메뉴로 바로가기</span></a>
[<a href="#lnb" tabindex="1"><span>메인 메뉴로 바로가기</span></a>, <a href="#main_content" tabindex="2"><span>본문으로 바로가기</span></a>, <a class="h_logo nclicks(STA.naver)" href="https://www.naver.com/"><span class="blind">NAVER</span></a>, <a class="h_news nclicks(STA.news)" href="/"><span class="blind">뉴스</span></a>]


(bs4.element.Tag, bs4.element.ResultSet, 232)

## **2 Focusing 객체의 상대경로**
- Focusing 객체가 참조하는 **href** url 경로를 추출 합니다
- 경로 내용은 **절대경로** 이외에 **상대경로** 값을 갖는다
- **상대경로** 는 현재 작업중인 **Url** 을 참조하여 **절대경로를 추출** 해야 한다

> **requests.compat.urljoin** (html.url, 추출한 상대경로)

In [5]:
# 특정한 class 를 활용한 객체내 속성값 추출
print(dom.find("", {"class":"h_logo"}).name, \
      dom.find("", {"class":"h_logo"}).text, \
      dom.find("", {"class":"h_logo"})["href"])

[_["class"] for _ in dom.find_all("a")   if _.has_attr("class")][:5]

a NAVER https://www.naver.com/


[['h_logo', 'nclicks(STA.naver)'],
 ['h_news', 'nclicks(STA.news)'],
 ['nclicks(STA.enter)'],
 ['nclicks(STA.sports)'],
 ['nclicks(STA.newsstand)']]

In [6]:
# 객체가 참조하는 href 추출하기 (절대경로 ,상대경로 가 추출)
[_["href"] for _ in dom.find_all("a")   if _.has_attr("href")][:5]

['#lnb',
 '#main_content',
 'https://www.naver.com/',
 '/',
 'https://entertain.naver.com/home']

In [7]:
# 작업중인 Url 절대주소와 결합하여 상대경로 완성하기
[requests.compat.urljoin(html.url, _["href"]) 
    for _ in dom.find_all("a")   
    if _.has_attr("href")][:5]

['https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=015&aid=0004180601#lnb',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=015&aid=0004180601#main_content',
 'https://www.naver.com/',
 'https://news.naver.com/',
 'https://entertain.naver.com/home']

## **3 DOM 자식의 탐색 (recursive)**
- **recursive** = **False** 자식 까지만 탐색
- **.find().find()** 메서드를 사용하여 수집한 자손단계를 특정한다

In [8]:
# .find().find_all() : 특정한 자손들의 내용을 모두 수집
dom.find("div").find_all("a")[:5], "///Next///",\
dom.find("div", {"class":"press_logo"}).find_all("a")[:5]

([<a href="#lnb" tabindex="1"><span>메인 메뉴로 바로가기</span></a>,
  <a href="#main_content" tabindex="2"><span>본문으로 바로가기</span></a>,
  <a class="h_logo nclicks(STA.naver)" href="https://www.naver.com/"><span class="blind">NAVER</span></a>,
  <a class="h_news nclicks(STA.news)" href="/"><span class="blind">뉴스</span></a>,
  <a class="nclicks(STA.enter)" href="https://entertain.naver.com/home"><img alt="TV연예" height="19" src="https://imgnews.pstatic.net/image/news/2017/10/snb_h_entertain.png" width="52"/></a>],
 '///Next///',
 [<a class="nclicks(atp_press)" href="http://www.hankyung.com/" target="_blank"><img alt="한국경제" class="" height="35" src="https://mimgnews.pstatic.net/image/upload/office_logo/015/2017/12/27/logo_015_38_20171227161527.png" title="한국경제"/></a>])

In [9]:
# recursive=False : 자식까지만 탐색
dom.find("div").find_all("a", recursive=False), "///Next///", \
dom.find("div", {"class":"press_logo"}).find_all("a")

([],
 '///Next///',
 [<a class="nclicks(atp_press)" href="http://www.hankyung.com/" target="_blank"><img alt="한국경제" class="" height="35" src="https://mimgnews.pstatic.net/image/upload/office_logo/015/2017/12/27/logo_015_38_20171227161527.png" title="한국경제"/></a>])

In [10]:
# limit=3 : 검색결과를 제한한다 (SQL Limit)
dom.find("div").find_all("a", limit=3), "///Next///", \
dom.find("div", {"class":"press_logo"}).find_all("a")

([<a href="#lnb" tabindex="1"><span>메인 메뉴로 바로가기</span></a>,
  <a href="#main_content" tabindex="2"><span>본문으로 바로가기</span></a>,
  <a class="h_logo nclicks(STA.naver)" href="https://www.naver.com/"><span class="blind">NAVER</span></a>],
 '///Next///',
 [<a class="nclicks(atp_press)" href="http://www.hankyung.com/" target="_blank"><img alt="한국경제" class="" height="35" src="https://mimgnews.pstatic.net/image/upload/office_logo/015/2017/12/27/logo_015_38_20171227161527.png" title="한국경제"/></a>])

In [11]:
# DOM 의 내용 간단하게 살펴보기
str(dom.contents[-1])[:1000]

'<html lang="ko">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta contents="always" name="referrer"/>\n<meta content="width=1106" name="viewport"/>\n<title>\'브렉시트 강경파\' 보리스 존슨 英 새 총리 오르다 : 네이버 뉴스</title>\n<meta content="한국경제 : 네이버뉴스" property="me2:post_tag"/>\n<meta content="한국경제" property="me2:category1"/>\n<meta content="세계" property="me2:category2"/>\n<meta content="https://imgnews.pstatic.net/image/015/2019/07/23/0004180601_001_20190724011504741.jpg" property="me2:image"/>\n<meta content="\'브렉시트 강경파\' 보리스 존슨 英 새 총리 오르다" property="og:title"/>\n<meta content="article" property="og:type"/>\n<meta content="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;oid=015&amp;aid=0004180601&amp;sid1=001" property="og:url"/>\n<meta content="https://imgnews.pstatic.net/image/015/2019/07/23/0004180601_001_20190724011504741.jpg" property="og:image"/>\n<meta content="영국의 새 총리에 ‘유럽의 트럼프’로 불리는 보리스 존슨 전 외무장관이 23일 선출됐다. 존슨 신임 총리가 유럽의 트럼프로 불리는 것은

In [12]:
# 현재 작업중인 Base Url
print(html.url)

# 객체들의 내용과 상대경로 결합하기
for _ in dom.find("div", {"id":"lnb"}).find_all("li"):
    print("{} : {}".format(_.text, requests.compat.urljoin(html.url, _.find("a")["href"])))

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=015&aid=0004180601
뉴스홈  : https://news.naver.com/main/home.nhn
속보 : https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=001
정치  : https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=100
경제  : https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=101
사회  : https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=102
생활/문화  : https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=103
세계 선택됨 : https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=104
IT/과학  : https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105
오피니언  : https://news.naver.com/main/opinion/home.nhn
포토  : https://news.naver.com/main/photo/index.nhn?mid=pho
TV  : https://news.naver.com/main/tv/index.nhn?mid=tvh
랭킹뉴스  : https://news.naver.com/main/ranking/popularDay.nhn?mid=etc&sid1=111


## **4 특정 Page DOM 작업하기 실습**
- 뉴스 페이지 DOM 작업하기
- 페이지 요소들의 Title.Text 와 참고 URL 절대경로와 결합

In [13]:
# 특정한 기사를 대상으로 작업 진행하기
# LG 디스플레이 실적기사 본문
url  = "https://news.naver.com/main/ranking/read.nhn?mid=etc&oid=025&aid=0002924369"
html = download('get', url)
type(html), html.url

(requests.models.Response,
 'https://news.naver.com/main/ranking/read.nhn?mid=etc&oid=025&aid=0002924369')

In [14]:
dom  = BeautifulSoup(html.text, "lxml")

dom.find("h3", {"id":"articleTitle"}).text, \
dom.find("h3", {"id":"articleTitle"}).find_next_sibling().find().text

('LG디스플레이, 2분기째 적자…LG폰 물량 확보도 ‘빨간 불’', '2019.07.23. 오후 5:08')

In [15]:
# Tree 관계를 활용한 Chain Method 로 추출
dom.find("h3", {"id":"articleTitle"}).find_parent().find_parent().find_next_sibling().find().text[:500]

'\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n             LG디스플레이가 올 2분기(4~6월) 부진한 성적표를 받아들었다. 2분기 연속적자일뿐더러 적자 폭도 올 1분기(1~3월) 대비 더욱 커졌다. 중국 디스플레이 업체의 공세로 인해 액정(LCD) 패널 가격이 내려간 탓이다. 회사는 LCD를 대체할 자발광소재 유기발광다이오드(OLED) 패널에 3조원을 추가 투자할 계획을 밝혔다.      23일 LG디스플레이는 올 2분기 매출 5조3534억원, 영업손실 3687억원을 기록했다고 공시했다. 시장 컨센서스(영업 손실 3000억원 안팎) 대비 부진한 성적이다. 1분기(1320억원)뿐 아니라 전년 같은 기간(지난해 2분기, 2881억원)과 비교해서도 적자 폭이 늘어났다. 실적 악화의 이유로 LG디스플레이는 “미ㆍ중 무역 전쟁 등 거시경제 우려가 확대됐고, 유통ㆍ세트업체들이 패널 구매를 보수적으로 전환해'

In [16]:
# Tag 추출 대상을 특정하지 않고서 추출하기
dom.find("", {"id":"articleBody"}).text[:500]

'\n\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n             LG디스플레이가 올 2분기(4~6월) 부진한 성적표를 받아들었다. 2분기 연속적자일뿐더러 적자 폭도 올 1분기(1~3월) 대비 더욱 커졌다. 중국 디스플레이 업체의 공세로 인해 액정(LCD) 패널 가격이 내려간 탓이다. 회사는 LCD를 대체할 자발광소재 유기발광다이오드(OLED) 패널에 3조원을 추가 투자할 계획을 밝혔다.      23일 LG디스플레이는 올 2분기 매출 5조3534억원, 영업손실 3687억원을 기록했다고 공시했다. 시장 컨센서스(영업 손실 3000억원 안팎) 대비 부진한 성적이다. 1분기(1320억원)뿐 아니라 전년 같은 기간(지난해 2분기, 2881억원)과 비교해서도 적자 폭이 늘어났다. 실적 악화의 이유로 LG디스플레이는 “미ㆍ중 무역 전쟁 등 거시경제 우려가 확대됐고, 유통ㆍ세트업체들이 패널 구매를 보수적으로 전환'

In [17]:
# 주의할 점!!
# bs4 객체는 .태그 대상이 없으면 NoneType 을 출력한다
# Focus 대상의 속성, 값을 필요시 NoneType 여부를 확인 후 진행하기 
type(dom.testPython)

NoneType

# **Crawling List**
## **1 참조 URL 목록 및 내용 수집기**
- **https://news.naver.com**
- Url Page 가 목록을 포함하는 경우, **개별 객체들의 URL** 추출하기
- 수집한 경로를 사용하여 **재귀적 함수를 활용한 내용 추출하기**

In [18]:
# Naver News 메인 페이지 크롤링
url  = "https://news.naver.com"
html = download('get', url)
dom  = BeautifulSoup(html.text, "lxml")
ul   = dom.find("", {"class":"hdline_article_list"})
[_.name   for _ in ul.find_all(recursive=False)]   # class="name"

['li', 'li', 'li', 'li', 'li', 'li']

In [19]:
# 개별 기사들 메모리에 저장하기
newsList = []
ul = dom.find("", {"class":"hdline_article_list"})

for i, _ in enumerate(ul.find_all(recursive=False)):
    print(_.find().find().text.strip())
    print(requests.compat.urljoin(html.url,  _.find().find()["href"]))
    news = BeautifulSoup(
        download("get",
            requests.compat.urljoin(html.url,  _.find().find()["href"])).text, "lxml")
    
    # print(news.find(attrs={"id":"articleBodyContents"}).text.strip())
    
    # NoneType 여부를 확인 (type(dom.testPython))
    if news.find( attrs={"id":"articleBodyContents"} ): 
        newsList.append( ( i+1, 
            news.find( attrs={"id":"articleBodyContents"} ).text.strip()))

나경원, 볼턴과 회동…"중러 영공침범 속 한미동맹 중요성 강조"(종합)
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0010977527
'반쪽 최고위' 재현된 바른미래당... 손학규 저격한 오신환
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=047&aid=0002234546
통상본부장 방미…“반도체 가격 급등 미국 등에 악영향”
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=056&aid=0010725196
'천안문 강경진압 주도' 리펑 별세…'학살자' 오명 꼬리표
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=104&oid=437&aid=0000215743
정부, 日 '백색국가서 한국 제외' 관련 브리핑[현장연결]
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=422&aid=0000384936

https://news.naver.com/


In [20]:
for _ in newsList:
    print(_[0], _[1][400:450])

1 등 엄중한 안보 현실에서 한미동맹의 중요성을 강조했다"고 소개했다.    그러면서 "일본의
2 역시 손 대표의 거취 문제가 달려 있다. 지도부 재신임 등을 담은 혁신안 상정 논란을 거부
3 체를 쓰는 모든 제품에까지 연결될 수 있는..."] 지금 당장은 삼성 하이닉스 같은 한국 
4 부터 10년동안 총리를 지냈습니다.천안문 사태 때는 강경 진압을 이끌고 시위대를 무력으로 
5 이 입법예고한 것에 대해 한국정부는 다시 한번 깊은 유감을 표합니다.금일 제출한 수출무역관


## **2 Crawling List 를 활용한 Focused**
```python
print(dom.find( attrs={ "id" : "results" }) ) # 반복적 작업시 세분화된 작업에 용이
print(dom.select_one( "#results" )) # CSS 선택자로 lxml 등 활용범위가 더 높음
```

- **find(),find_all()** 과 **select(),select_one()** 의 구분
- 둘다 비슷하지만 약간 문법이 다르고, 둘 중 1개로만 작업해도 무관하다
- CSS Selector
> h1 { color : red , font-size: 10;}

In [21]:
# CSS 셀렉터를 활용한 Focusing
len(dom.select(".hdline_article_list")), \
dom.select(".hdline_article_list > li > div > a")[0].text.strip()

(1, '나경원, 볼턴과 회동…"중러 영공침범 속 한미동맹 중요성 강조"(종합)')

In [22]:
# 목록의 제목만 추출
for _ in dom.select(".hdline_article_list > li > div > a.lnk_hdline_article"):
    print(_.text.strip())

나경원, 볼턴과 회동…"중러 영공침범 속 한미동맹 중요성 강조"(종합)
'반쪽 최고위' 재현된 바른미래당... 손학규 저격한 오신환
통상본부장 방미…“반도체 가격 급등 미국 등에 악영향”
'천안문 강경진압 주도' 리펑 별세…'학살자' 오명 꼬리표
정부, 日 '백색국가서 한국 제외' 관련 브리핑[현장연결]



In [23]:
# 목록의 제목과 본문의 추출
for i, _ in enumerate(dom.select(".hdline_article_list a.lnk_hdline_article")[:-1]):
    url  = requests.compat.urljoin(html.url, _["href"])
    print(i, _.text.strip())
    news = BeautifulSoup(download("get", url).text, "lxml")
    print(news.select_one("#articleBodyContents").text.strip().splitlines()[0])

0 나경원, 볼턴과 회동…"중러 영공침범 속 한미동맹 중요성 강조"(종합)
// flash 오류를 우회하기 위한 함수 추가
1 '반쪽 최고위' 재현된 바른미래당... 손학규 저격한 오신환
// flash 오류를 우회하기 위한 함수 추가
2 통상본부장 방미…“반도체 가격 급등 미국 등에 악영향”
// flash 오류를 우회하기 위한 함수 추가
3 '천안문 강경진압 주도' 리펑 별세…'학살자' 오명 꼬리표
동영상 뉴스
4 정부, 日 '백색국가서 한국 제외' 관련 브리핑[현장연결]
동영상 뉴스


<br></br>
# **Selenium**
## **1 Selenium DOM Driver 생성 및 활용**
! pip install selenium
- http://example.webscraping.com/places/default/search
- headless 를 막는 경우도 있어서 경우에 따라 옵션을 적용하기

In [24]:
# 크롤링 실습예제 사이트 크롤링 (한계가 존재)
url = "http://example.webscraping.com/places/default/search"
dom = BeautifulSoup(download("get", url).text, "lxml")

print(dom.find(attrs={"id":"results"}))
print(dom.select_one("#results")) # CSS 문법을 활용가능해 이를 더 추천

<div id="results">
</div>
<div id="results">
</div>


In [25]:
# Selenium의 실행
# '/usr/lib/chromium-browser/chromedriver' 경로는 각자 환경에 맞게 설정값 적용이 필요
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver')

# 현재 활성화 중인 핸들 확인하기
driver.window_handles

['CDwindow-3FB97B855A505921441A5F6A6E41FDFA']

In [27]:
# Index 값을 활용한 Handle 윈도우를 특정
# 핸들링 윈도우를 특정합니다 ex)[0] : 현재 활성화 창
driver.switch_to_window(driver.window_handles[0])
# driver.close()

# 빈 Browser 에 URL 추가하기
driver.get(url)

# Driver 내용을 BS4 객체로 변환 및 활용
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select_one("#results")

<div id="results">
</div>

In [28]:
driver.find_element_by_xpath("//input[@id='search_term']")

# input 에 입력된 내용의 확인
driver.find_element_by_xpath("//input[@id='search_term']").get_attribute("value")

<selenium.webdriver.remote.webelement.WebElement (session="9c035647a66a0a57f17b657ed6a839b4", element="0.5590135167668271-1")>

In [30]:
# <input> 태그에 내용 입력
driver.find_element_by_xpath("//input[@id='search_term']").clear() # 입력된 내용의 제거
driver.find_element_by_css_selector("#search_term").send_keys("korea")  # 새로운 값 입력
print(driver.find_element_by_xpath("//input[@id='search_term']").get_attribute("value"))

# click() 이벤트 실행
driver.find_element_by_tag_name("form").find_element_by_xpath("//input[@type='submit']").click()
driver.find_element_by_css_selector("#results").text

korea


''

In [31]:
# DOM 내용의 확인 과정이 필요 (try/ except 난립을 막을 수 있다!!!!, 처리속도도 빠름)
driver.find_element_by_css_selector("#results").is_displayed()

True

In [32]:
driver.close()

## **2 Ajax 통신 내용의 확인**
- **XML, JSON** 통신 내용 경로를 활용한 자료수집

In [33]:
# html JSON 데이터 송수신 내용의 활용
# 해당 경로를 GET 방식으로 수집하기
url = "http://example.webscraping.com/places/ajax/search.json"
params = {
    "search_term": "",
    "page_size":10,
    "page":0,
}
params["search_term"] = "china"
html = download("get", url, params)
html.json()

{'records': [{'pretty_link': '<div><a href="/places/default/view/China-47"><img src="/places/static/images/flags/cn.png" /> China</a></div>',
   'country': 'China',
   'id': 4593503}],
 'num_pages': 1,
 'error': ''}

In [34]:
import re
for _ in html.json()["records"]:
    print(_["pretty_link"]); print('*' * 20)
    
    group = re.findall('<a href="(.+)?">',  _["pretty_link"])
    print(group); print('*' * 20)
    print(requests.compat.urljoin(html.url, group[0]))  # 이미지 상대주소를 절대주소로 변환
    
# img = download("get", "http://example.webscraping.com/")
# img

<div><a href="/places/default/view/China-47"><img src="/places/static/images/flags/cn.png" /> China</a></div>
********************
['/places/default/view/China-47']
********************
http://example.webscraping.com/places/default/view/China-47


<br></br>
# **Cookie & Session 활용**
- http://pythonscraping.com/pages/cookies/welcome.php

## **1 requests 를 활용한 Cookie 수집 및 분석하기**
- requests.Session() 객체를 활용한 Cookie 분석 
- Cookies 정보내용 살펴보기

```json
Request URL: http://pythonscraping.com/pages/cookies/welcome.php
Request Method: POST
Status Code: 200 OK
Remote Address: 198.27.68.184:80
Referrer Policy: no-referrer-when-downgrade
username: erwerqew
password: password
```

In [35]:
# BeautifulSoup 를 활용한 수집
# url = "http://pythonscraping.com/pages/cookies/login.html"
url  = "http://pythonscraping.com/pages/cookies/welcome.php"
data = {
    "username": "erwerqew",
    "password": "password"    
}
html = download("post", url, data=data)
html.text

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [36]:
# requests Session() 을 활용한 수집
session = requests.Session()
html    = session.post(url, data)
print(html.text)
session.cookies


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


<RequestsCookieJar[Cookie(version=0, name='loggedin', value='1', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='username', value='erwerqew', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False)]>

## **2 Selenium 을 활용한 Cookie 수집 및 분석하기**
- driver() 를 활용한 수집 후
- requests.Session() 객체를 활용한 Cookie 분석을 활용 

In [37]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver')
url    = "http://pythonscraping.com/pages/cookies/login.html"
driver.get(url)
driver.get_cookies()

[]

In [38]:
# Selenium 이벤트 적용
driver.find_element_by_name("username").send_keys("testset")
driver.find_element_by_name("password").send_keys("password")
driver.find_element_by_css_selector("input[type=submit]").click()

# 비어있는 Session() 을 생성 후, driver 내용 입력하기
session = requests.Session()
for _ in driver.get_cookies():
    print(_["name"], _["value"])
    session.cookies.set(_["name"], _["value"])

driver.close()
print(session.cookies.keys())
session.cookies

username testset
loggedin 1
['loggedin', 'username']


<RequestsCookieJar[Cookie(version=0, name='loggedin', value='1', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='username', value='testset', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

<br></br>
# **실습하기**
- 인터파크 로그인 및 좌석예매하기 by Selenium
- 네이버 뉴스 수집 후 Text 로 저장

In [39]:
# 뉴스 Page Bs4 객체로 저장하기
from bs4 import BeautifulSoup
url = "https://news.naver.com"
dom = BeautifulSoup(download("get", url).text, "lxml")

# Bs4 객체 내 제목의 수집
for _ in dom.select("ul.hdline_article_list  a.lnk_hdline_article"):
    if len(_["href"]) >1: # if _.has_attr("href"):
        print(_.text.strip())

나경원, 볼턴과 회동…"중러 영공침범 속 한미동맹 중요성 강조"(종합)
'반쪽 최고위' 재현된 바른미래당... 손학규 저격한 오신환
통상본부장 방미…“반도체 가격 급등 미국 등에 악영향”
'천안문 강경진압 주도' 리펑 별세…'학살자' 오명 꼬리표
정부, 日 '백색국가서 한국 제외' 관련 브리핑[현장연결]


In [40]:
# 개별 뉴스들의 Link 주소 수집하기
css_select = "ul.hdline_article_list  a.lnk_hdline_article,  ul.mlist2.no_bg li > a"
newsList = [requests.compat.urljoin(url, _["href"])  
            for _ in dom.select(css_select)
            if len(_["href"]) >1]
print(len(newsList), newsList[0])

35 https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0010977527


In [41]:
# aid 인덱스 값을 활용한 저장파일명 만들기
for _ in newsList[:5]:
    print(_.split("&")[-1][4:]+ ".txt")

0010977527.txt
0002234546.txt
0010725196.txt
0000215743.txt
0000384936.txt


In [42]:
%%time
# aid 파일명 생성 및 Link 수집결과 저장하기
for _ in newsList:
    fileName = _.split("&")[-1][4:]+ ".txt"
    dom      = BeautifulSoup(download("get", _).text, "lxml")
    content  = dom.select_one("#articleBodyContents").text.strip()
    with open("./News/" + fileName, "w", encoding="utf-8") as f:
        f.write(content)
        
print("News Crawling is Saved.. ")

News Crawling is Saved.. 
CPU times: user 2.27 s, sys: 103 ms, total: 2.37 s
Wall time: 4.13 s
